# Single neuron linear and logistic regression in Keras

This uses `bikes` data 


In [1]:
import pandas as pd
import numpy as np

In [7]:
%pip install tensorflow
%pip install keras
%pip install sklearn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a las

In [8]:
from tensorflow import keras
from keras import layers

In [9]:

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


## Linear Regression

In [12]:
bikes = pd.read_csv('/Users/blove/PycharmProjects/deepLearning/Data/bikes.csv') ### your path will depend on where you put the file.

predictors = ['temp_feel', 'humidity', 'windspeed']

X = bikes[predictors]
y = bikes['rides']

scl = StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2)

X_train = scl.fit_transform(X_train)
X_test = scl.transform(X_test)



#### Using Keras Sequential API.


In [13]:

lin_reg = keras.models.Sequential(
    [
        layers.Dense(1, activation= None, input_shape=(X_train.shape[1],))
    ]
)

lin_reg.compile(
    optimizer='sgd',
    loss=  'mse',
    metrics=['RootMeanSquaredError']
)

/opt/homebrew/Cellar/jupyterlab/4.4.7/libexec/lib/python3.13/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
lin_reg.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4 (16.00 B)

 Trainable params: 4 (16.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
lin_reg.fit(X_train, y_train, 
         epochs = 50, 
         validation_data=(X_test, y_test)
         )

Another way to do it is by using `.add` to add layers.

In [ ]:
lin_reg = keras.Sequential()
lin_reg.add(layers.Dense(1, activation=None))


lin_reg.compile(
    optimizer='sgd',
    loss=  'mse',
    metrics=['RootMeanSquaredError']
)

In [ ]:
lin_reg.fit(X_train, y_train, 
         epochs = 50, 
         validation_data=(X_test, y_test)
         )

#### Coefficients, in order, and the bias.

In [ ]:
lin_reg.evaluate(X_test, y_test, verbose=1)

In [ ]:

lin_reg.layers[0].get_weights()

#### rmse and R^2

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

results = lin_reg.evaluate(X_test, y_test)

rmse = results[1]
r2 = r2_score(y_test, lin_reg.predict(X_test))


print("RMSE:", rmse)
print("Rsq:", r2)


#### Now sklearn

We get identical or very nearly identical

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

rmse = np.sqrt(mean_squared_error(y_test, lin_reg.predict(X_test)))
r2 = lin_reg.score(X_test, y_test)

print("RMSE = ", rmse)
print("Rsq = ", r2)


In [ ]:
print("coeffs:", lin_reg.coef_)
print("intercept:", lin_reg.intercept_)


## Logistic Regression

We use the `cancer` data set

In [ ]:
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()

In [ ]:
cancer.keys()

In [ ]:
X = cancer.data
y = cancer.target
scl = StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = scl.fit_transform(X_train)
X_test = scl.transform(X_test)


#### Using Keras

In [ ]:
log_reg = keras.models.Sequential(
    [
        layers.Dense(1, activation= 'sigmoid', input_shape=(X_train.shape[1],))
    ]
)

log_reg.compile(
    optimizer='sgd',
    loss=  'binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
log_reg.fit(X_train, y_train, 
         epochs = 50, 
         validation_data=(X_test, y_test)
         )


In [ ]:
log_reg = keras.Sequential()
log_reg.add(layers.Dense(1, activation="sigmoid"))

log_reg.compile(
    optimizer='sgd',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [ ]:
log_reg.evaluate(X_test, y_test, verbose=0)

#### Coefficients

In [ ]:
for predictor, coeff in list(zip(X.columns, log_reg.layers[0].get_weights()[0])):
    print(f"{predictor}: {coeff}")

#### Weights

In [ ]:
log_reg.layers[0].get_weights()[0]

#### Weights

In [ ]:
log_reg.layers[0].get_weights()[1]

### Now try adding a few more layers  